In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from networkx import NetworkXError
from tqdm.notebook import tqdm

from quapopt import ancillary_functions as anf

from quapopt.data_analysis.data_handling import (CoefficientsType,
                                                 CoefficientsDistribution,
                                                 CoefficientsDistributionSpecifier,
                                                 HamiltonianModels)
from quapopt.hamiltonians.generators import build_hamiltonian_generator, ERDOS_RENYI_TYPES

### Generate random Hamiltonians and save them to files

* We will generate random Hamiltonians from a given model (Sherrington-Kirkpatrick in this case) and save them to files.
* We can generate various classes.
* Instances can be generated on the fly or read from file.
* We can also solve the Hamiltonian classicaly, to get the ground state energy and the highest energy.
* We can also save the Hamiltonian to file.



In [3]:
# Choose hamiltonia model
hamiltonian_model = HamiltonianModels.SherringtonKirkpatrick
#choose system sizes to generate
full_noq_range = [15]
#choose seeds for instances generation
full_seeds_range = list(range(0, 3))

coefficients_type = CoefficientsType.DISCRETE
coefficients_distribution = CoefficientsDistribution.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}

coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

arguments_dict_builder = {}

localities = None
erdos_renyi_type = None
#In some cases, we are required to specify whether to add local fields (localities = (1,2)) or just 2-body interactions (localities = (2,))
if hamiltonian_model in [HamiltonianModels.SherringtonKirkpatrick, ]:
    localities = (1,2)
elif hamiltonian_model in [HamiltonianModels.RegularGraph]:
    localities = (2,)
elif hamiltonian_model in [HamiltonianModels.ErdosRenyi]:
    localities = (1, 2)
    #Which type of erdos-renyi graph to use
    erdos_renyi_type=ERDOS_RENYI_TYPES.Gnp
elif hamiltonian_model in [HamiltonianModels.MaxCut]:
    pass
elif hamiltonian_model in [HamiltonianModels.MAX2SAT]:
    pass
elif hamiltonian_model in [HamiltonianModels.Wishart]:
    pass
elif hamiltonian_model in [HamiltonianModels.LABS]:
    #LABS is deterministic, so no seed is needed
    full_seeds_range = [0]
else:
    raise ValueError(f"Unknown hamiltonian model: {hamiltonian_model}")
generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier,
                                                         erdos_renyi_type=erdos_renyi_type)

print(full_noq_range)


[15]


In [4]:
solve_hamiltonians = True
solver_kwargs = {'solver_name': "BURER2002",
                 'solver_timeout': 1}

save_hamiltonians = True


for number_of_qubits in tqdm(full_noq_range):
    anf.cool_print("NUMBER OF QUBITS", number_of_qubits)

    for seed_hamiltonian in full_seeds_range:
        print("INSTANCE:", seed_hamiltonian)

        arguments_instance = {'number_of_qubits': number_of_qubits,
                              'seed': seed_hamiltonian,
                              'read_from_drive_if_present': True}
        if hamiltonian_model in [HamiltonianModels.SherringtonKirkpatrick, ]:
            pass
        elif hamiltonian_model in [HamiltonianModels.RegularGraph]:
            graph_degree = 3
            arguments_instance['graph_degree'] = graph_degree
        elif hamiltonian_model in [HamiltonianModels.ErdosRenyi]:
            average_degree = number_of_qubits // 2
            #edge_probability = average_degree / (number_of_qubits - 1)
            edge_probability = 0.01

            arguments_instance['p_or_M'] = edge_probability
        elif hamiltonian_model in [HamiltonianModels.MaxCut]:
            average_degree = number_of_qubits // 2
            edge_probability = average_degree / (number_of_qubits - 1)
            arguments_instance['p_or_M'] = edge_probability
        elif hamiltonian_model in [HamiltonianModels.MAX2SAT]:
            clause_density = 1.1
            arguments_instance['clause_density'] = clause_density

        elif hamiltonian_model in [HamiltonianModels.WishartPlantedEnsemble]:
            pass

        try:
            cost_hamiltonian = generator_cost_hamiltonian.generate_instance(**arguments_instance)
        except NetworkXError as e:
            print("NetworkXError", e)
            continue

        if solve_hamiltonians:
            #This will write known solutions if they already exist in instance attributes (if not, it doesn't do anything)
            cost_hamiltonian.write_solutions_to_file()
            #This solving is required to get lowest energy state and highest energy state
            cost_hamiltonian.solve_hamiltonian(both_directions=True, #both directions means whether to also find highest energy state
                                               solver_kwargs=solver_kwargs)

        ground_state_energy = cost_hamiltonian.ground_state_energy
        highest_energy = cost_hamiltonian.highest_energy

        anf.cool_print("GROUND STATE ENERGY", ground_state_energy)
        anf.cool_print("HIGHEST ENERGY", highest_energy)
        if save_hamiltonians:
            cost_hamiltonian.write_to_file()



  0%|          | 0/1 [00:00<?, ?it/s]

NUMBER OF QUBITS 15
INSTANCE: 0
GROUND STATE ENERGY np.float64(-40.0)
HIGHEST ENERGY np.float64(44.0)
Hamiltonian exists, not overwriting.
INSTANCE: 1
GROUND STATE ENERGY np.float64(-46.0)
HIGHEST ENERGY np.float64(40.0)
Hamiltonian exists, not overwriting.
INSTANCE: 2
GROUND STATE ENERGY np.float64(-42.0)
HIGHEST ENERGY np.float64(46.0)
Hamiltonian exists, not overwriting.


In [5]:
print("Okay, then")

Okay, then
